#Task 
Predict ว่าคนหนึ่งๆจะเรียนจบคอร์สหนึ่งๆหรือไม่
#Dataset (df2)
  Feature ทุุุกอย่างเหมือนอีกแบบหนึ่งยกเว้น
*   avg_score 
*   quiz_submit
*   quiz_open

ผมไม่ได้เอา record ทั้งหมดมาทำ feature แต่ตัดเอา record ครึ่งนึงมาทำ feature






In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV




Using TensorFlow backend.


In [0]:
#df2=pd.read_csv("dfhalf.csv")
df2=pd.read_csv("20191112_CUMOOC_Data/df13.csv")
df2=df2.drop(columns=["all_score","nub","cv_uid","cv_cid","UC"])
df2


NameError: ignored

In [0]:
df2 = shuffle(df2)
X_train2=pd.DataFrame(df2)
Y_train2=X_train2["label"]
X_train2=X_train2.drop(columns=["label"])
Layer1DF=pd.DataFrame(df2)


#ลองเอา dataset มา Train ก่อน

In [0]:
lg=LogisticRegression()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)


0.8611769513690233
0.7626506024096386 0.4925111845944369 0.598510814324548


In [0]:
def build_nn():
  Kclass=Sequential()
  Kclass.add(Dense(15,activation='tanh',input_dim=6))
  Kclass.add(Dropout(0.2))
  Kclass.add(Dense(1,activation='sigmoid'))
  Kclass.compile(optimizer="adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
  return Kclass
KRlayer1=KerasClassifier(build_fn=build_nn,epochs=100,batch_size=10)
KRlayer1.fit(X_train2[:85000],Y_train2[:85000])
k=KRlayer1.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=KRlayer1.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





85000/85000 [==============================] - 13s 148us/step - loss: 0.2659 - acc: 0.8766
Epoch 2/100
85000/85000 [==============================] - 12s 139us/step - loss: 0.2347 - acc: 0.8898
Epoch 3/100
85000/85000 [==============================] - 12s 138us/step - loss: 0.2302 - acc: 0.8939
Epoch 4/100
85000/85000 [==============================] - 12s 138us/step - loss: 0.2295 - acc: 0.8934
Epoch 5/100
85000/85000 [==============================] - 12s 137us/step - loss: 0.2270 - acc: 0.8951
Epoch 6/100
85000/85000 [==============================] - 12s 138us/step - loss: 0.2272 - acc: 0.8948
Epoch 7/100
85000/85000 [==============================] - 12s 139us/step - loss: 0.2261 - acc: 0.8962
Epoch 8/100
85000/85000 [==============================]

In [0]:
lg=lgbm.LGBMClassifier()
lg.fit(X_train2[:85000],Y_train2[:85000])
k=lg.score(X_train2[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train2[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)

0.9312627707396812
0.8030488873313475 0.8914608052907994 0.8449483775811211


In [0]:
X_train3=X_train2.drop(columns=["G1","G2","G3"])
lg=lgbm.LGBMClassifier()
lg.fit(X_train3[:85000],Y_train2[:85000])
k=lg.score(X_train3[85000:], Y_train2[85000:])
print(k)
lgy=lg.predict(X_train3[85000:])
tn, fp, fn, tp = confusion_matrix(Y_train2[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)

0.9270535349407437
0.7891109579600276 0.8908772612332231 0.8369118318867064


# ที่ต่างจากแบบแรกคือ LogisticRegression มี f1 score ที่ลดลงมากเหลือประมาณ 50% และทุกโมเดลมี accuracy ลดลง

#Stacking

In [0]:
LRlayer1=LogisticRegression()
LRlayer1.fit(X_train2,Y_train2)
out=LRlayer1.predict_proba(X_train2)
Layer1DF["LRProb"]=out[:,1]
Layer1DF

,label,enroll_count,complete_count,quiz_submit,quiz_open,avg_score,did_watch,G1,G2,G3,LRProb
90845,0,7,3,1,2,0.000000,1,0,1,0,0.405607
107929,0,8,2,0,0,0.000000,0,0,1,0,0.023758
105919,0,5,3,0,1,0.000000,1,0,1,0,0.431703
98670,0,5,0,0,1,0.000000,0,1,0,0,0.018819
78284,1,1,1,16,10,0.857143,0,0,0,0,0.866308
...,...,...,...,...,...,...,...,...,...,...,...
19548,0,8,1,0,0,0.000000,0,0,1,0,0.016881
85810,0,0,0,0,1,0.000000,1,1,0,0,0.337558
99614,1,1,1,0,1,0.000000,1,0,0,0,0.387808
13579,0,0,0,0,1,0.000000,0,1,0,0,0.036337


In [0]:
lg=lgbm.LGBMClassifier()
lg.fit(X_train2,Y_train2)
out=lg.predict_proba(X_train2)
Layer1DF["LGProb"]=out[:,1]

In [0]:
ada=AdaBoostClassifier()
ada.fit(X_train2,Y_train2)
out=ada.predict_proba(X_train2)
Layer1DF["ADAProb"]=out[:,1]

In [0]:
ext=ExtraTreesClassifier()
ext.fit(X_train2,Y_train2)
out=ext.predict_proba(X_train2)
Layer1DF["EXTProb"]=out[:,1]

In [0]:
kn=KNeighborsClassifier()
kn.fit(X_train2,Y_train2)
out=kn.predict_proba(X_train2)
Layer1DF["KNProb"]=out[:,1]

In [0]:
rf=RandomForestClassifier()
rf.fit(X_train2, Y_train2)
out=rf.predict_proba(X_train2)
Layer1DF["RFProb"]=out[:,1]
#OO=rf.feature_importances_.reshape(1,6)
#for i in range(OO.shape[1]):
#  IM[i]+=OO[0,i]


In [0]:
xgb = XGBClassifier()
xgb.fit(X_train2, Y_train2)
print(xgb.score(X_train2,Y_train2))
out=xgb.predict_proba(X_train2)
Layer1DF["XGProb"]=out[:,1]

0.924947474193843


In [0]:
def build_nn():
  Kclass=Sequential()
  Kclass.add(Dense(15,activation='tanh',input_dim=9))
  Kclass.add(Dropout(0.2))
  Kclass.add(Dense(1,activation='sigmoid'))
  Kclass.compile(optimizer="adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
  return Kclass
KRlayer1=KerasClassifier(build_fn=build_nn,epochs=100,batch_size=10)
KRlayer1.fit(X_train2,Y_train2)
out=KRlayer1.predict_proba(X_train2)
Layer1DF["KRProb"]=out[:,1]




Epoch 1/100





109470/109470 [==============================] - 14s 125us/step - loss: 0.2503 - acc: 0.8812
Epoch 2/100
109470/109470 [==============================] - 15s 134us/step - loss: 0.2203 - acc: 0.8955
Epoch 3/100
109470/109470 [==============================] - 14s 124us/step - loss: 0.2141 - acc: 0.8997
Epoch 4/100
109470/109470 [==============================] - 13s 123us/step - loss: 0.2109 - acc: 0.9019
Epoch 5/100
109470/109470 [==============================] - 13s 122us/step - loss: 0.2098 - acc: 0.9025
Epoch 6/100
109470/109470 [==============================] - 13s 123us/step - loss: 0.2092 - acc: 0.9029
Epoch 7/100
109470/109470 [==============================] - 14s 126us/step - loss: 0.2092 - acc: 0.9041
Epoch 8/100
109470/109470 [==============================] - 13s 121us/step - loss: 0.2089 - acc: 0.9030
Epoch 9/100
109470/109470 [==============================] - 14s 126us/step - loss: 0.2076 - acc: 0.9031
Epoch 10/100
109470/109470 [===================

In [0]:
#IM=[[-val,key] for key,val in IM.items()]
#IM.sort()
#print(IM)
#Layer1DF=Layer1DF.drop(columns=["complete_count","did_watch"])
#ayer1DF[Layer1DF["label"]==1]
#Layer1DF

In [0]:
Layer1DF.to_csv("dflayer1half_sex.csv",index=False)

In [0]:
Layer1DF=pd.read_csv("dflayer1half_sex.csv")

In [0]:
lg=lgbm.LGBMClassifier()
Y=Layer1DF["label"]
MM=Layer1DF.drop(columns=["label"])
lg.fit(MM[:85000],Y[:85000])
k=lg.score(MM[85000:], Y[85000:])
k2=lg.score(MM[:85000], Y[:85000])
print(k2,k)
lgy=lg.predict(MM[85000:])
tn, fp, fn, tp = confusion_matrix(Y[85000:],lgy).ravel()
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)

0.9560235294117647 0.9487127094401308
0.8623352165725047 0.8973153047227121 0.8794775761067896


In [0]:
lg=LogisticRegression()
Y=OK["label"]
MM=OK.drop(columns=["label"])
lg.fit(MM[:80000],Y[:80000])
k=lg.score(MM[80000:], Y[80000:])
print(k)
lgy=lg.predict(MM[80000:])
tn, fp, fn, tp = confusion_matrix(Y[80000:],lgy).ravel()
print(tn,fp,fn,tp)
precision=tp/(tp+fp)
recall=tp/(tp+fn)
f1=2*precision*recall/(precision+recall)
print(precision,recall,f1)

# หลังการ stack จะได้ accuracy ประมาณ 95% น้อยกว่าแบบแรก (97%)
